In [1]:
# Using test data to build/prototype-out a ML model
    # Reshaping, encoding, normalization
    # Input, layer, and output specification
    # Force keras/tensorflow to use GPU backend

In [16]:
# Imports
import pandas as pd
import numpy as np
import json

# Pymongo import for connection to local client DB
import pymongo
from pymongo import MongoClient

# ML Imports 
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras import backend as K

# Import to check check for GPU availability for tensorflow backend
from tensorflow.python.client import device_lib

In [17]:
# Mongodb connection for live feed of data into model
    # Implementation after verification of test data -> model prototype input layer working

In [18]:
# Verify GPU availability for tensorflow backend
print(device_lib.list_local_devices())
print("==============================================")
print(K.tensorflow_backend._get_available_gpus())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1160176137233909572
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 9222031934
locality {
  bus_id: 1
  links {
  }
}
incarnation: 3867392018611426427
physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1"
]
['/job:localhost/replica:0/task:0/device:GPU:0']
